# Load example data

This loads example output files which can be used to create visualizations while the rest of the software is under construction. 

It will remove all previous outputs from the database.

It requires the `sample_data_set` folder to be in the root, specified below.

It requires at least one user to be created.

In [ ]:
import os
from analysis import models
from myuser.models import User

In [ ]:
example_root = os.path.abspath('/Users/shapiroaj4/Desktop/sample_data_set')
assert os.path.exists(example_root)

In [ ]:
# get first user; you must create one if needed
user = User.objects.all()[0]

In [ ]:
print(user.email)

In [ ]:
# delete all the old stuff
models.FeatureList.objects.all().delete()
models.Analysis.objects.all().delete()
models.FeatureListCountMatrix.objects.all().delete()

In [ ]:
fl = models.FeatureList.objects.create(
    owner=user,
    name='demo1',
    genome_assembly=1,
    stranded=True,
)
fl.dataset.name = os.path.join(example_root, 'unt1hr.obsTSS.bed')
fl.save()

In [ ]:
an = models.Analysis.objects.create(
    owner=user,
    name='demo1',
    genome_assembly=1,
    feature_list=fl,
    sort_vector=None,
)

In [ ]:
ds = models.EncodeDataset.objects.all()[0]
mat_dir = os.path.join(example_root, 'individual_matrices')
for fn in os.listdir(mat_dir):
    cm = models.FeatureListCountMatrix.objects.create(
        feature_list=fl,
        dataset=ds,
        matrix=os.path.join(mat_dir, fn)
    )
    
    adm = models.AnalysisDatasets.objects.create(
        analysis=an,
        dataset=ds,
        display_name=os.path.splitext(os.path.basename(fn)),
        count_matrix=cm
    )
    
    an.analysisdatasets_set.add(adm)

In [ ]:
dcm = models.DatasetCorrelationMatrix.objects.create(
    analysis=an,
    matrix=os.path.join(example_root, 'analysis_output.json')
)

In [ ]:
from django.core.urlresolvers import reverse
print(reverse('analysis:visual_testing', args=[an.id]))